In [3]:
import torch.nn as nn
import cv2
from torchvision import transforms
import torch
import torchvision
import numpy as np
import ds_ear
import glob
from PIL import Image
from matplotlib import image
from matplotlib import pyplot

CATEGORIES = ["Konrad", "Falco"]
RESIZE_Y = 150
RESIZE_X = 100
DATA_TEST_FOLDER = "../test/*png"


def get_data(folder):
    img_array = []
    img_array_resized = []
    files = glob.glob (folder)
    for idx, f in zip(range(len(files)),files):
        image = cv2.imread(f)
        img_array.append (image)
        img_array_resized.append(cv2.resize(img_array[idx],(RESIZE_Y,RESIZE_X)))
    return np.asarray(img_array_resized)


model = torch.load('./class_sample/model.pt')



In [7]:
data = get_data(DATA_TEST_FOLDER)
data_tensor = torch.from_numpy(data)
data_tensor = data_tensor.permute(0, 3, 1, 2)
if get_devide() == 'cpu':
    data_tensor = data_tensor.type('torch.FloatTensor')
else:
    data_tensor = data_tensor.type('torch.cuda.FloatTensor')


In [8]:
NUMBER_AUTHORIZED = int(.7*len(data_tensor))
#NUMBER_AUTHORIZED = 7
with torch.no_grad():
	pred = model(data_tensor)
	pred = torch.softmax(pred, 1)
	pred = pred.cpu().numpy()

classes_ = np.argmax(pred, 1)
print(pred)
print(classes_)
counts = np.bincount(classes_)
if np.max(counts) > NUMBER_AUTHORIZED:
	print("Welcome to your save room " + CATEGORIES[np.argmax(counts)] + "!")
else: 
	print("Authentification Failed! You got no acces rights!")

[[1.1676162e-04 2.2581856e-04 5.5376968e-05 ... 2.3213437e-05
  4.6539740e-04 2.2324240e-04]
 [1.9089397e-03 8.4995141e-04 3.2910591e-03 ... 3.4838170e-04
  4.7138617e-03 8.0978433e-03]
 [2.3707566e-03 5.3619174e-04 9.6994609e-04 ... 1.1815072e-04
  1.3865153e-03 3.5390852e-03]
 ...
 [2.8774858e-04 2.4712755e-04 9.3374954e-05 ... 4.5862533e-05
  6.5038283e-04 5.6786888e-04]
 [2.2746357e-03 3.9359811e-04 7.9304841e-04 ... 2.4490370e-04
  2.7393422e-03 4.6773031e-03]
 [3.1858210e-03 9.6882385e-04 1.2686818e-03 ... 4.9018534e-04
  3.2439390e-03 1.0540009e-02]]
[ 5 96 25  5  5 96 96  5 26 96  5 96  5  5 26  5]
Authentification Failed! You got no acces rights!


In [ ]:
# TODO: Hier muss ein ganzes Array an Bilder geladen werden!!!   

image = Image.open("../test/konrad080.png")

transform=torchvision.transforms.Compose([
    torchvision.transforms.Resize((RESIZE_Y, RESIZE_X)),
    torchvision.transforms.Lambda(lambda x: x.convert('RGB')),
    torchvision.transforms.ToTensor(),

    torchvision.transforms.Normalize(
        [0.49139968, 0.48215841, 0.44653091],
        [0.24703223, 0.24348513, 0.26158784]
    )

])
image_transformed = transform(image)
image_transformed = image_transformed.reshape(-1, RESIZE_Y, RESIZE_X, 1)
image_transformed = image_transformed.permute(3, 0, 1, 2)
image_transformed = image_transformed.type('torch.cuda.FloatTensor')
